In [7]:
import os
from langchain_core.messages import HumanMessage
from langgraph.graph import END, MessageGraph

from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq


from dotenv import load_dotenv
load_dotenv()
groq_api_key = os.getenv("GROQ_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")
LANGCHAIN_API_KEY = os.getenv("LANGCHAIN_API_KEY")
LANGCHAIN_ENDPOINT = os.getenv("LANGCHAIN_ENDPOINT")
LANGCHAIN_PROJECT = os.getenv("LANGCHAIN_PROJECT")  




In [8]:
model =  ChatGroq(temperature=1, model_name="llama3-70b-8192", api_key=groq_api_key)


In [3]:

from langchain.globals import set_debug
from langchain.globals import set_verbose

set_verbose(False)
set_debug(False)


In [4]:

graph = MessageGraph()

graph.add_node("oracle", model)
graph.add_edge("oracle", END)

graph.set_entry_point("oracle")

runnable = graph.compile()

runnable.get_graph().print_ascii()



+-----------+  
| __start__ |  
+-----------+  
      *        
      *        
      *        
  +--------+   
  | oracle |   
  +--------+   
      *        
      *        
      *        
 +---------+   
 | __end__ |   
 +---------+   


In [76]:
runnable.invoke(HumanMessage("What is 1 + 1?"))


[chain/start] [chain:LangGraph] Entering Chain run with input:
[inputs]
[chain/start] [chain:LangGraph > chain:__start__] Entering Chain run with input:
[inputs]
[chain/end] [chain:LangGraph > chain:__start__] s] Exiting Chain run with output:
[outputs]
[chain/start] [chain:LangGraph > chain:oracle] Entering Chain run with input:
[inputs]
[llm/start] [chain:LangGraph > chain:oracle > llm:ChatGroq] Entering LLM run with input:
{
  "prompts": [
    "Human: What is 1 + 1?"
  ]
}
[llm/end] [chain:LangGraph > chain:oracle > llm:ChatGroq] [3.94s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "1 + 1 is 2.",
        "generation_info": {
          "finish_reason": "stop",
          "logprobs": null
        },
        "type": "ChatGeneration",
        "message": {
          "lc": 1,
          "type": "constructor",
          "id": [
            "langchain",
            "schema",
            "messages",
            "AIMessage"
          ],
          "kwargs": {
 

[HumanMessage(content='What is 1 + 1?', id='aa752065-0527-4299-bc69-cb73c334330c'),
 AIMessage(content='1 + 1 is 2.', response_metadata={'token_usage': {'completion_time': 0.027, 'completion_tokens': 8, 'prompt_time': 0.053, 'prompt_tokens': 18, 'queue_time': None, 'total_time': 0.08, 'total_tokens': 26}, 'model_name': 'llama3-70b-8192', 'system_fingerprint': 'fp_c1a4bcec29', 'finish_reason': 'stop', 'logprobs': None}, id='run-b39b8378-ed42-4e55-bfbe-2af7a2ee2aef-0')]

In [67]:
from langchain_core.tools import tool
from langgraph.prebuilt import ToolNode

@tool
def multiply(first_number: int, second_number: int):
    """Multiplies two numbers together."""
    return first_number * second_number

In [68]:
model_with_tools = model.bind_tools([multiply])


In [69]:
builder = MessageGraph()

builder.add_node("oracle", model_with_tools)

tool_node = ToolNode([multiply])
builder.add_node("multiply", tool_node)

builder.add_edge("multiply", END)

builder.set_entry_point("oracle")

In [70]:
from typing import Literal

from typing import List, Literal
from langchain_core.messages import BaseMessage

def router(state: List[BaseMessage]) -> Literal["multiply", "__end__"]:
    tool_calls = state[-1].additional_kwargs.get("tool_calls", [])
    if len(tool_calls):
        return "multiply"
    else:
        return "__end__"

builder.add_conditional_edges("oracle", router)



In [71]:
runnable = builder.compile()

runnable.invoke(HumanMessage("What is 123 * 456?"))

[chain/start] [chain:LangGraph] Entering Chain run with input:
[inputs]
[chain/start] [chain:LangGraph > chain:__start__] Entering Chain run with input:
[inputs]
[chain/end] [chain:LangGraph > chain:__start__] s] Exiting Chain run with output:
[outputs]
[chain/start] [chain:LangGraph > chain:oracle] Entering Chain run with input:
[inputs]
[llm/start] [chain:LangGraph > chain:oracle > llm:ChatGroq] Entering LLM run with input:
{
  "prompts": [
    "Human: What is 123 * 456?"
  ]
}


Error in ConsoleCallbackHandler.on_tool_end callback: AttributeError("'int' object has no attribute 'strip'")


[llm/end] [chain:LangGraph > chain:oracle > llm:ChatGroq] [1.80s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "",
        "generation_info": {
          "finish_reason": "tool_calls",
          "logprobs": null
        },
        "type": "ChatGeneration",
        "message": {
          "lc": 1,
          "type": "constructor",
          "id": [
            "langchain",
            "schema",
            "messages",
            "AIMessage"
          ],
          "kwargs": {
            "content": "",
            "additional_kwargs": {
              "tool_calls": [
                {
                  "id": "call_gv29",
                  "function": {
                    "arguments": "{\"first_number\":123,\"second_number\":456}",
                    "name": "multiply"
                  },
                  "type": "function"
                }
              ]
            },
            "response_metadata": {
              "token_usage": {
               

[HumanMessage(content='What is 123 * 456?', id='14537f36-42f8-4672-aa3c-e25d6e4df3a1'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_gv29', 'function': {'arguments': '{"first_number":123,"second_number":456}', 'name': 'multiply'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_time': 0.219, 'completion_tokens': 123, 'prompt_time': 0.335, 'prompt_tokens': 1135, 'queue_time': None, 'total_time': 0.554, 'total_tokens': 1258}, 'model_name': 'mixtral-8x7b-32768', 'system_fingerprint': 'fp_c5f20b5bb1', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-62246cc3-d15d-4951-acae-a98bdd80a7f9-0', tool_calls=[{'name': 'multiply', 'args': {'first_number': 123, 'second_number': 456}, 'id': 'call_gv29'}]),
 ToolMessage(content='56088', name='multiply', id='acd80e1d-4fbf-432a-b4df-c3157dbf855d', tool_call_id='call_gv29')]

In [72]:
runnable.get_graph().print_ascii()



          +-----------+      
          | __start__ |      
          +-----------+      
                 *           
                 *           
                 *           
            +--------+       
            | oracle |       
            +--------+       
           ..         ..     
         ..             ..   
        .                 .. 
+----------+                .
| multiply |              .. 
+----------+            ..   
           **         ..     
             **     ..       
               *   .         
            +---------+      
            | __end__ |      
            +---------+      


In [ ]:
from langchain_core.runnables.graph import CurveStyle, NodeColors, MermaidDrawMethod
from IPython.display import display, HTML, Image

display(
    Image(
        runnable.get_graph().draw_mermaid_png(
            draw_method=MermaidDrawMethod.API,
        )
    )
)

In [116]:
the_question="""create 10 unique names that incorporate elements of Greek mythology and playful twists, you can follow these steps:
Combine with Modern Elements: Mix these mythological names with modern or technological terms that would cover any service from outsourcing 
cleaning for hotel rooms down to air traffic control, network engineers, software developers, kitchen cheffs, restaurant waiters, etc. 
Playful Twists and Modifications: Modify the names slightly to make them unique or to fit better with the modern elements you've chosen. 
Ensure Uniqueness: Make sure each name is distinct from the others by varying the mythological component or the modern term and joyfull
and easy to remember"""
answer = runnable.invoke(HumanMessage(the_question))



[chain/start] [chain:LangGraph] Entering Chain run with input:
[inputs]
[chain/start] [chain:LangGraph > chain:__start__] Entering Chain run with input:
[inputs]
[chain/end] [chain:LangGraph > chain:__start__] s] Exiting Chain run with output:
[outputs]
[chain/start] [chain:LangGraph > chain:oracle] Entering Chain run with input:
[inputs]
[llm/start] [chain:LangGraph > chain:oracle > llm:ChatGroq] Entering LLM run with input:
{
  "prompts": [
    "Human: create 10 unique names that incorporate elements of Greek mythology and playful twists, you can follow these steps:\nCombine with Modern Elements: Mix these mythological names with modern or technological terms that would cover any service from outsourcing \ncleaning for hotel rooms down to air traffic control, network engineers, software developers, kitchen cheffs, restaurant waiters, etc. \nPlayful Twists and Modifications: Modify the names slightly to make them unique or to fit better with the modern elements you've chosen. \nEnsure

In [135]:
print(answer[1].content)


Here are 10 unique names that incorporate elements of Greek mythology and playful twists:

1. **ZeusByte** (Zeus + Byte) - perfect for a software development company that's powerful and efficient.
2. **AuroraNet** (Aurora + Network) - suitable for a network engineering firm that brings light and connectivity to its clients.
3. **HermesDeliver** (Hermes + Deliver) - ideal for a food delivery or logistics company that's fast and reliable.
4. **AthenaTech** (Athena + Tech) - great for a technology consulting firm that's wise and innovative.
5. **PoseidonPro** (Poseidon + Pro) - suitable for a professional cleaning service that's powerful and thorough, like the sea god.
6. **DianaDish** (Diana + Dish) - perfect for a restaurant or catering company that's swift and delicious, like the goddess of the hunt.
7. **HephaestusHelp** (Hephaestus + Help) - ideal for a technical support or IT services company that's skilled and reliable, like the Greek god of the forge.
8. **ArtemisAir** (Artemis + 

In [119]:
from langgraph.prebuilt import ToolNode
from langchain_community.tools.tavily_search import TavilySearchResults

tools = [TavilySearchResults(max_results=2)]
tool_node = ToolNode(tools)

modelwith_tools = model.bind_tools(tools)

In [121]:
from typing import TypedDict, Annotated

def add_messages(left: list, right: list):
    """Add-don't-overwrite."""
    return left + right

class AgentState(TypedDict):

    messages: Annotated[list, add_messages]
    
from typing import Literal

# Define the function that determines whether to continue or not
def should_continue(state: AgentState) -> Literal["action", "__end__"]:
    messages = state['messages']
    last_message = messages[-1]
    # If the LLM makes a tool call, then we route to the "action" node
    if last_message.tool_calls:
        return "action"
    # Otherwise, we stop (reply to the user)
    return "__end__"


# Define the function that calls the model
def call_model(state: AgentState):
    messages = state['messages']
    response = modelwith_tools.invoke(messages)
    # We return a list, because this will get added to the existing list
    return {"messages": [response]}

In [122]:
from langgraph.graph import StateGraph, END
# Define a new graph
workflow = StateGraph(AgentState)

# Define the two nodes we will cycle between
workflow.add_node("agent", call_model)
workflow.add_node("action", tool_node)

# Set the entrypoint as `agent`
# This means that this node is the first one called
workflow.set_entry_point("agent")

# We now add a conditional edge
workflow.add_conditional_edges(
    # First, we define the start node. We use `agent`.
    # This means these are the edges taken after the `agent` node is called.
    "agent",
    # Next, we pass in the function that will determine which node is called next.
    should_continue,
)

# We now add a normal edge from `tools` to `agent`.
# This means that after `tools` is called, `agent` node is called next.
workflow.add_edge('action', 'agent')

# Finally, we compile it!
# This compiles it into a LangChain Runnable,
# meaning you can use it as you would any other runnable
app = workflow.compile()

In [123]:
app.get_graph().print_ascii()

        +-----------+           
        | __start__ |           
        +-----------+           
               *                
               *                
               *                
          +-------+             
          | agent |             
          +-------+             
          *        ..           
        **           ..         
       *               .        
+--------+         +---------+  
| action |         | __end__ |  
+--------+         +---------+  


In [ ]:
from langchain_core.runnables.graph import CurveStyle, NodeColors, MermaidDrawMethod
from IPython.display import display, HTML, Image

display(
    Image(
        app.get_graph().draw_mermaid_png(
            draw_method=MermaidDrawMethod.API,
        )
    )
)

In [126]:
from langchain_core.messages import HumanMessage

inputs = {"messages": [HumanMessage(content="Search in internet for this question: " + the_question)]}
results = app.invoke(inputs)

In [136]:
print(results['messages'][3].content)

Here are 10 unique names that incorporate elements of Greek mythology with modern twists:

1. **Zeusra**: A combination of Zeus, the king of the gods, and "ra" from radar, suggesting advanced technology and surveillance systems.
2. **AthenaCode**: A mix of Athena, the goddess of wisdom, and "code," implying expertise in software development and programming.
3. **PoseiTech**: A blend of Poseidon, the god of the sea, and "tech," suggesting innovative solutions for marine conservation and oceanography.
4. **HeraHub**: A combination of Hera, the queen of the gods, and "hub," implying a central connection point for network engineers and IT specialists.
5. **DemosKitchen**: A mix of Demos, the Greek word for "people," and "kitchen," suggesting a culinary expertise that caters to diverse tastes and cuisines.
6. **ArtemisAir**: A combination of Artemis, the goddess of the hunt, and "air," implying efficient and agile air traffic control systems.
7. **Achilleanalytics**: A blend of Achilles, th

In [141]:
response_message = model.invoke(
    "In bash, how do I list all the text files in the current directory that have been modified in the last month?"
)

print(response_message.content)
print(response_message.response_metadata)

You can use the `find` command to achieve this. Here's an example:
```
find . -type f -name "*.txt" -mtime -30 -print
```
Let's break it down:

* `.` is the current directory.
* `-type f` specifies that we only want to consider files (not directories).
* `-name "*.txt"` matches files with the `.txt` extension (adjust this pattern to match your specific needs).
* `-mtime -30` selects files that have been modified within the last 30 days (i.e., in the last month).
* `-print` simply prints the names of the matching files.

When you run this command, you should see a list of text files in the current directory that have been modified in the last month.

If you want to list files recursively (i.e., including subdirectories), add the `-recursive` option:
```
find . -type f -name "*.txt" -mtime -30 -recursive -print
```
Note that `-mtime` uses the file's modification time, which is the time the file's contents were last modified. If you want to consider the file's access time (i.e., when the 